In [1]:
%pip install -q langchain-elasticsearch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from uuid import uuid4
from elasticsearch import Elasticsearch
from langchain_core.documents import Document
from langchain_elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import ElasticSearchBM25Retriever

In [2]:
embeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': False},
)

C:\Users\andre\AppData\Local\Temp\ipykernel_13356\527724830.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(


In [4]:
elastic_store = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="tyuiu_index",
    embedding=embeddings,
    es_user="elastic",
    es_password="password",
)

In [5]:
import os


def read_files_in_directory(directory_path: str) -> str:
    files = []
    if not os.path.exists(directory_path):
        return
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        files.append(file_path)
    return files


directory_path = r'C:\Users\andre\AdvancedRAG\static\chanks'
files = read_files_in_directory(directory_path)

In [6]:
def get_file_name(file_path: str) -> str:
    return file_path.split("\\")[-1]

In [7]:
documents = [
    Document(
        page_content=open(file, 'r', encoding='utf-8').read(),
        metadata={"source": get_file_name(file)}
    )
    for file in files
]
uuids = [str(uuid4()) for _ in range(len(documents))]

In [24]:
# elastic_store.add_documents(documents=documents, ids=uuids)

['4e1c5faa-ca80-4404-8228-7c80950628dd',
 '065d1715-76a8-42b5-ad05-df515defa8a3',
 '77094c96-1825-49c5-9291-b8bbe8847b4f',
 '675fb0ce-fd69-42e8-b892-d1a57bd5feee',
 'ebc25145-b350-4cf9-b138-296adc528143',
 'a76f866a-f005-4f4e-a384-18a01be5fcfc',
 'cd49c0e8-eb86-40db-8e8c-d206985711a6',
 '72a79a05-5dd3-4bf8-9cec-48f92eeb3da5',
 'd4655c9c-51ea-4a74-8250-0d102ce2c649',
 '6ed88792-0638-4dcc-b087-f3a41d6b2ac0',
 '999373ac-fe20-4fdb-a157-d0693eb85469',
 '0fba1210-c955-4e30-afbe-a1fc111bff7d',
 '43c51f36-f8c4-40dc-ad12-6d21836efd35',
 '3c35ca76-73e0-4756-8f95-0845ccb69206',
 '44abccbc-3e77-42fe-990c-6f00b13603ca',
 '9f9fa314-a6a2-442a-89bb-2b5955ac0f88',
 'ab78d70e-87bf-48e8-9774-c064e5486f20',
 'a7bf9346-3641-4e65-b838-f2224dbb0b0b',
 'f608e506-0ca2-4269-82b1-258a655d4218',
 '71985a3b-0703-4658-85ce-17fedb1bc69b',
 '09729438-40f7-466a-a2df-8ce94748c2bf',
 '6561091a-4119-4141-9949-02331cd711f6',
 '9b4da1d3-00c8-46e2-8b90-7109da4d7c31',
 'aa369d1c-a8f5-498a-aade-4d02566aadd1',
 'fb2c7fc2-2c46-

In [21]:
docs = elastic_store.similarity_search("какой минимальный проходной балл на направление архитектура")
docs

[Document(metadata={'source': 'Количество организаций, специальностей.txt'}, page_content='Количество организаций, специальностей и (или) направлений подготовки для одновременного поступления на обучение по программам бакалавриата и программам специалитета\nПредельное количество организаций, в которые поступающий вправе одновременно поступать на обучение по программам бакалавриата и программам специалитета включая Университет, составляет 5.Предельное количество специальностей и (или) направлений подготовки, по которым поступающий вправе одновременно участвовать в конкурсе по программам бакалавриата и программам специалитета в Университете составляет 5.\nВ Университете поступающий может одновременно поступать на обучение по различным конкурсным группам в рамках каждой специальности, каждого направления подготовки.\n'),
 Document(metadata={'source': 'Доп.испытания-2.txt'}, page_content='Для поступления на направление “Архитектура” (07.03.01) нужно сдать два дополнительных вступительных и

In [8]:
elastic_client = Elasticsearch(
    hosts="http://localhost:9200",
    basic_auth=("elastic", "password")
)

bm25_retriever = ElasticSearchBM25Retriever(
    client=elastic_client, 
    index_name="docs-tyuiu-index",
    search_kwargs={"k": 2}
)

In [23]:
# bm25_retriever.add_texts([document.page_content for document in documents])

['50b584a3-74ad-49bb-b1e3-54eddad1ee50',
 '49df2b77-14ef-4adf-a406-ab7b1026645d',
 'eb6d65bb-1eb6-4826-939e-fe3e149e079c',
 '5c78ca10-c2af-4627-89ed-de6fe6d15984',
 'cebbfcb2-416a-4320-8abb-3cab578a8812',
 '464c1e36-f602-46f0-b109-017e65770ac5',
 'fa574f41-8558-481e-878a-06dff56647da',
 'b84ba7de-974a-4474-9ecd-a06fbe5d100d',
 '070deb9a-ee4a-49c3-b6fb-507825a6c9bf',
 'a971c06b-6913-4098-a026-37092a04ffbc',
 'afb9dfc0-fb56-4fb2-94fd-0749bba4eae2',
 'eea8c978-2402-400b-b1eb-c644c0063a9a',
 '67f74264-3917-4b41-908b-c596fadbebf0',
 'c307741d-f574-4516-97d6-d7df521ba432',
 '65137210-ec57-48ad-9c62-b8eb943b3c47',
 '9ffeedc9-b239-4111-b841-d6916ad71e10',
 'a5ebda5c-60bc-424a-8f04-a3f5f031d125',
 '627dfd54-bce1-4b7d-84dc-cb611c80a965',
 '2493de92-d1dd-425f-9e67-959a0eb0b800',
 '3d610f3f-ca16-4a22-993f-9cf40dedccb0',
 '90251b95-ef7f-413e-b104-df23d1eae0ad',
 '72e581e4-7b8e-4154-9850-cac5c9a41d48',
 '9e1aa422-44cb-4309-b3a3-f68f5a43b901',
 'b753afe7-402f-4f20-9b7c-8ed2f9168a81',
 'f45dfc5a-2691-

In [9]:
vector_store_retriever = elastic_store.as_retriever(search_kwargs={"k": 2})

In [10]:
from langchain.retrievers import EnsembleRetriever

In [11]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_store_retriever, bm25_retriever],
    weights=[0.6, 0.4]
)

In [14]:
docs = ensemble_retriever.invoke("Алгоритм поступления")
for doc in docs:
    print(doc)

page_content='Количество организаций, специальностей и (или) направлений подготовки для одновременного поступления на обучение по программам бакалавриата и программам специалитета
Предельное количество организаций, в которые поступающий вправе одновременно поступать на обучение по программам бакалавриата и программам специалитета включая Университет, составляет 5.Предельное количество специальностей и (или) направлений подготовки, по которым поступающий вправе одновременно участвовать в конкурсе по программам бакалавриата и программам специалитета в Университете составляет 5.
В Университете поступающий может одновременно поступать на обучение по различным конкурсным группам в рамках каждой специальности, каждого направления подготовки.
' metadata={'source': 'Количество организаций, специальностей.txt'}
page_content='Эти правила определяют порядок приема в Тюменский индустриальный университет и его филиалы на программы бакалавриата, специалитета и магистратуры в 2025 году для граждан РФ